In [1]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
import os


D:\python\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
# 데이터 로드
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# 데이터 행개수 확인 (총 100,000 개)
print (("Read {0} labeled train reviews, {1} labeled test reviews, "+ '\n' +"and {2} unlabeled reviews").format(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, 
and 50000 unlabeled reviews


In [9]:
model = Word2Vec.load("300features_40minwords_10context")

In [10]:
model.syn1neg.shape

(16490, 300)

In [11]:
model["kitchen"].shape  # kitchen이 one-hot이 아닌 distributed representation 된 matrix 

(300,)

In [12]:
np.divide(np.ones((300,5),dtype="float32"),2)

array([[ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       ..., 
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5]], dtype=float32)

word2vec에서 각 리뷰안의 각 단어들의 embedding matrix(가중치 matrix)를 모두 합해 평균내어 각 리뷰마다 하나의 대표값을 만들었는데, 이는 평균값이 나중에는 피쳐벡터중에서 하나의 피쳐가 된다. 즉 해당 리뷰에 대한 고유의 피쳐값을 구하는 이 과정은, CNN의 convolution layer에서 흑백이미지의 피쳐를 뽑은 후 average pooling 하는 과정과 상당히 유사함을 알수가 있다. CNN에서 하나의 피쳐맵은 input 사진의 특정부분을 잘 구분하는 가중곱의 합이 되어진 convolved 피쳐는 word2vec에서 input과 input vector를 곱한 방식과 유사하고 (정확히 같지는 않다.그냥 가중곱을 하여 다른 공간에 사영시킨다는 측면에서는 유사하다는 것이 포인트) 다음 average pooling 을 하여 기존 피쳐맵을 줄이는 과정은 평균을 통해 피쳐안의 피쳐를 뽑겠다는 말인데 , 이것이 word2vec에서도 한 리뷰안에서 embedding된 각 단어들의 matrix들의 평균을 내는 것과 유사하다. 

In [13]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality
##min_word_count = 40   # Minimum word count
#num_workers = 4       # Number of threads to run in parallel
#context = 10          # Context window size
#downsampling = 1e-3   # Downsample setting for frequent words

In [14]:
def makeFeatureVec(words, model, num_features):
    # 주어진 문장의 단어벡터들의 평균을 계산하는 함수
    
    # 피쳐벡터 초기화(속도를 위해)
    featureVec = np.zeros((num_features,),dtype="float32")
    
    nwords = 0.
    
    # 모델의 vocabulary에 있는 단어들의 이름이 담긴 Index2word를 속도를 위해 set으로 바꾼다.
    index2word_set = set(model.wv.index2word)
    
    # 각 리뷰에서 각 단어들에 대해 반복문을 돌린다,
    # 만약 단어가 모형의 vocabulary안에 있다면 기존 피쳐벡터에 이 단어의 피쳐벡터(엠베딩된 단어 matrix)를 더한다.
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    # 평균벡터을 얻기위해 단어들의 수로 피쳐벡터를 나눈다. 
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [16]:
def getAvgFeatureVecs(reviews, model, num_features):
    # 한 세트의 리뷰가 주어졌을때 (각 리뷰는 단어 리스트임, 결국 전체 구조는 리스트 속 리스트),
    # 각 리뷰에 대한 평균피쳐벡터를 계산하여 2차원 numpy array를 반환해라
    
    # counter 초기화
    counter = 0.
    
    # 속도를 위해, 미리 2차원 numpy array 할당.
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # 모든 리뷰마다 루프문 돌기
    for review in reviews:
       
       # 매 1000번째 리뷰마다 상태메세지 프린트
        if counter%1000. == 0.:
            print ("Review %d of %d" % (counter, len(reviews)))
       
        # 평균피쳐벡터들을 생성하는 makeFeatureVec 함수 호출
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
    
        # 카운터 1씩 증가.
        counter = counter + 1.
        
    return reviewFeatureVecs

In [17]:
def review_to_wordlist( review, remove_stopwords=False ):
    # document를 list화
    # 불용어를 없애는 것은 선택, 보통 높은 질의 워드벡터를 얻기 위해 불용어를 제거하지는 않음.
    
    # HTML 함수 지우기
    review_text = BeautifulSoup(review).get_text()
      
    # 글자가 아닌것들 없애기 []안의 ^는 not과 같음.
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 소문자화 후 띄어쓰기를 기준으로 모두 나눔.
    words = review_text.lower().split()
    
    # 불용어 제거
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 단어 리스트 반환
    return(words)

In [18]:
def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append(review_to_wordlist(review, remove_stopwords=True ))
    return clean_reviews


In [19]:
# ****************************************************************
# 위 함수들을 사용해, 트레이닝과 테스트셋을 위한 평균벡터들을 계산
# stop word를 제거하는것 잊지 말기

print ("Creating average feature vecs for train reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )


print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist( review, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features )

D:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file D:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [20]:
# 100개의 나무를 이용해 훈련셋을 랜덤포레스트에 적합.
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# 테스트 결과 할당
result = forest.predict(testDataVecs)

# 테스트 결과 추출
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [25]:
result_train = forest.predict(trainDataVecs)
train_output = pd.DataFrame(data = {"id":train["id"], "sentiment":result_train})

In [26]:
train_output.head()

,id,sentiment
0,"""5814_8""",1
1,"""2381_9""",1
2,"""7759_3""",0
3,"""3630_4""",0
4,"""9495_8""",1


In [27]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(train_output["sentiment"], train["sentiment"])

1.0